In [ ]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.vision.all import *

# Dogs and Cats to Pet Breeds

In our very first model, we learned how to classify dogs versus cats. Here, we will use the same dataset to work on a much more challenging problem: figuring out what breed of pet is shown in each image.

## Data Preparation
In real life, we often start with some dataset that we know nothing about. We then have to figure out how it is put together, how to extract the data we needed from it, and what that data looks like. Therefore, we will be solving these problems in practice, including all the intermediate steps required to understand the data and test our modelling as we go.

In [ ]:
# Downloading the Pet Image Dataset
path = untar_data(URLs.PETS)

# Setting the base path to the pet images directory we have just downloaded
Path.BASE_PATH = path

To understand how to extract the breed of each pet from each image, we will need to understand how this data is structured.

Data is usually provided in one of these two ways:
- Individual files representing items of data, such as text documents of images, possibly organised into folders or with filenames representing information about those items.
- A table of data where each row is an item which may include filenames providing a connection between the data in the table and data in other formats, such as text documents and images.

To see what is in our dataset, we can use the `ls` method provided by fastai.

In [ ]:
path.ls()

We can see that our dataset provides us with *images* and *annotations* directories. The website for the dataset tells us that the *annotations* directory contains information about where the pets are, rather than what they are. In this chapter, we will be doing classification, not localisation. Therefore, we will ignore the *annotations* directory.

Let's have a look inside the *images* directory.

In [ ]:
(path/"images").ls()

Most functions and methods in fastai that return a collection use a class called `L`, an enhanced version of the ordinary Python `list` type, with added convenience for common operations. 

For instance, when we display an object of class `L`, the first thing that is shown is the number of items in the collection, prefixed with a `#`. The list is also suffixed with an ellipsis, meaning that only the first few items are displayed.

By examining the filenames, we can see how they appear to be structured. Each filename consists of the pet breed, followed by an underscore, a number and finally the file extension (petbreed_XXX.jpg).
We will therefore need to create a piece of code that extracts the breen from a single `Path`. We do have to be careful to not make too many assumptions at this point however, as you may notice that some of the pet breeds contain multiple words, so we cannot simply break at the first `_` character that we find.

To allow us to test our code, let's pick out the first filename from our dataset.

In [ ]:
fname = (path/"images").ls()[0]; fname

The "best" way to extract information from strings like this is to use a *regular expression* (aka. regex). This is a special string, written in the regular expression language, which specifies a general rule for deciding if another string passes a test, and also possibly for extracting a particular part or parts out of that other string.

In this case, we need a regular expression that extracts the pet breed from the filename. The best way to start is just try it against on example at first.

In [ ]:
# Using the findall method to try a regular expression against the filename of the fname object
re.findall(r'(.+)_\d+.jpg', fname.name)

The above regular expression extracts all the characters leading up to the last underscore character, as long as the subsequent characters are numerical digits and then the JPEG file extension.

After confirming that the regex works for the example, we can use it for the whole dataset. Fortunately, fastai comes with many classes to help with labelling. For labelling with regular expressions, we can use the `RegexLabeller` class.

In [ ]:
# Using the data block API
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items = get_image_files,
                 splitter = RandomSplitter(seed=42),
                 get_y = using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms = Resize(460),
                 batch_tfms = aug_transforms(size=224, min_scale=0.75))

# Creating a DataLoaders object
dls = pets.dataloaders(path/'images')

One important piece of the above `DataBlock` call that we haven't seen before is in the two lines:
``` 
    item_tfms = Resize(460),
    batch_tfms = aug_transforms(size=224, min_scale=0.75)
```

These lines implement a fastai data augmentation strategy which we call *presizing*. This is a particular way to do image augmentation that is designed to minimise data destruction while maintaining good performance.

### Presizing

Our images need to have the same dimensions so that they can be collated into tensors to be passed to the GPU. We also want to minimise the number of distinct augmentation computations we perform. This suggests that we should compose our augmentation transforms into fewer transforms (to reduce the number of computations and the number of lossy operations) and transform the images into uniform sizes (for more efficient processing on the GPU).

Presizing adopts two strategies:
1. Resize images to relatively "large" dimensions - that is, dimensions significantly larger than the target training dimensions
2. Compose all of the common augmentation operations (including a resize to the final target size) into one, and perform the combined operation on the GPU only once at the end of processing, rather than performing the operations individually and interpolating multiple times.

The first step, the resize, creates images large enough that they have spare margin to allow further augmentation transforms on their inner regions without creating empty zones. It works by resizing to a square, using a large crop size. On the training set, the crop area is chosen randomly, and the size of the crop is selected to cover the entire width or height of the image, whichever is smaller.

In the second step, the GPU is used for all data augmentation, and all of the potentially destructive operations are done together, with a single interpolation at the end.

To implement this process in fastai, we use `Resize` as an item transform with a large size, and `RandomResizeCrop` as a batch transform with a smaller size. `RandomResizeCrop` will be added for you if you include the `min_scale` parameter in your `aug_transforms` function. Alternatively, you can use `pad` or `squish` instead of `crop` for the initial `Resize`.

In practice, using presizing significantly improves the accuracy of models, and often results in speedups too.

### Checking and Debugging a DataBlock

Writing a `DataBlock` is just like writing a blueprint. You will get an error message if you have a syntax error somewhere in your code, but you have no guarantee that your template is going to work on your data source as you intend. Before training a model, you should always check your data by using the `show_batch` method.

Take a look at each image, and check that each one seems to have the correct label for that breed of pet. Often, data scientists work with data where they are not as familiar as domain experts may be.



In [ ]:
dls.show_batch(nrows=1, ncols=3)

If you made a mistake while building your `DataBlock`, it is very likely you won't se it before this step. To debug this, it is useful to use the `summary` method which will attempt to create a batch from the source you give it, with a lot of details. If it fails, you will see exactly at which point the error happens, and the library will try to give you some help.

In [ ]:
pets.summary(path/"images")

We can see exactly how we gathered the data and split it, how we went from a filename to a sample (the tuple (image, category)), then what item transforms were applied and how it collated those samples in a batch.

Once you think your data looks right, it is recommended that the next step should be using it to train a simple model. For this initial test, we'll use the same simple model we used previously.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)